### Connect to database

In [20]:
%pwd

'/Users/kimmywong/Downloads/EDS_213/bren-eds213-data/database'

In [27]:
import duckdb
import pandas as pd
conn = duckdb.connect("database.db")

In [22]:
cur = conn.cursor()

### Step 1: Create a query that shows scientific names and species code for species with egg data.

In [23]:
cur.execute("""SELECT DISTINCT Code, Scientific_name FROM Species 
    INNER JOIN Bird_nests
    ON Species.Code = Bird_nests.Species
    INNER JOIN Bird_eggs
    ON Bird_eggs.Nest_ID = Bird_nests.Nest_ID
    ORDER BY Scientific_name""")
cur.fetchall()

[('rutu', 'Arenaria interpres'),
 ('dunl', 'Calidris alpina'),
 ('wrsa', 'Calidris fuscicollis'),
 ('sepl', 'Charadrius semipalmatus'),
 ('reph', 'Phalaropus fulicarius'),
 ('amgp', 'Pluvialis dominica'),
 ('bbpl', 'Pluvialis squatarola')]

### Step 2: After you’ve connected to the database and created a cursor c, iterate over the species like so:
species_query = """SELECT Code, Scientific_name FROM..."""
for row in cur.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    # query egg data for that species (step 3)
    # compute statistics and print results (step 4)


In [25]:
species_query = """SELECT DISTINCT Code, Scientific_name FROM Species 
    INNER JOIN Bird_nests
    ON Species.Code = Bird_nests.Species
    INNER JOIN Bird_eggs
    ON Bird_eggs.Nest_ID = Bird_nests.Nest_ID
    ORDER BY Scientific_name"""

for row in cur.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]

### Step 3: You will need to construct a query that gathers egg data for a given species, one species at a time; the species code will be a parameter to that query.



In [28]:
# Define species query from above
egg_query = """SELECT Width*Width*Length AS Volume, Species FROM Bird_eggs
    JOIN Bird_nests
    ON Bird_eggs.Nest_ID = Bird_nests.Nest_ID
    WHERE Bird_nests.Species = ?"""

# Iterate over species
for row in cur.execute(species_query).fetchall():
    species_code = row[0]
    scientific_name = row[1]
    egg_query = egg_query
    df = pd.read_sql(egg_query, conn, params=[species_code])
    print(df)

          Volume Species
0   18129.257812    rutu
1   27114.382812    rutu
2   30465.349609    rutu
3   38728.300781    rutu
4   31784.414062    rutu
5   35203.085938    rutu
6   34760.351562    rutu
7   35976.867188    rutu
8   35303.023438    rutu
9   37496.710938    rutu
10  36470.546875    rutu
11  36400.875000    rutu
12  18129.257812    rutu
13  27114.382812    rutu
14  30465.349609    rutu
15  38728.300781    rutu
16  34270.027344    rutu
17  32679.935547    rutu
18  37211.968750    rutu
19  42001.929688    rutu
20  44033.710938    rutu
21  45660.441406    rutu
22  46688.285156    rutu
          Volume Species
0   22976.552734    dunl
1   23460.568359    dunl
2   23576.367188    dunl
3   23206.162109    dunl
4   23200.333984    dunl
5   23714.867188    dunl
6   19850.609375    dunl
7   20749.957031    dunl
8   22288.044922    dunl
9   22131.791016    dunl
10  22265.214844    dunl
          Volume Species
0   20778.173828    wrsa
1   20857.626953    wrsa
2   20141.640625    wrsa


/var/folders/gr/jlw8fkqn023fr8q1p0l6lpq80000gn/T/ipykernel_19461/716735640.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(egg_query, conn, params=[species_code])
/var/folders/gr/jlw8fkqn023fr8q1p0l6lpq80000gn/T/ipykernel_19461/716735640.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(egg_query, conn, params=[species_code])
/var/folders/gr/jlw8fkqn023fr8q1p0l6lpq80000gn/T/ipykernel_19461/716735640.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(egg_query, con

### Step 4: Finally, and still within your loop, you’ll want to compute statistics and print out the results:


In [30]:
for row in cur.execute(species_query).fetchall():
    species_code = row[0]
    scientific_name = row[1]
    egg_query = egg_query
    df = pd.read_sql(egg_query, conn, params=[species_code])
    cv = round(df.Volume.std()/df.Volume.mean()*100, 2)
    print(f"{scientific_name} {cv}%")

Arenaria interpres 21.12%
Calidris alpina 5.46%
Calidris fuscicollis 16.77%
Charadrius semipalmatus 8.99%
Phalaropus fulicarius 4.65%
Pluvialis dominica 19.88%
Pluvialis squatarola 6.94%


/var/folders/gr/jlw8fkqn023fr8q1p0l6lpq80000gn/T/ipykernel_19461/425590428.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(egg_query, conn, params=[species_code])


In [32]:
# Close the connection
conn.close()